## Load Required Library

In [83]:
import pandas as pd
import os
import json
import re
import numpy as np
import random
import string
import pickle
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import pos_tag
import gensim
from gensim import corpora, models, similarities
from sklearn.metrics.pairwise import cosine_similarity

from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

In [84]:
import os

## Data Preparation

### 01-Read Raw Data

In [85]:
path = ''
data = pd.read_csv('tagged_posts.tsv',sep='\t')
len(data)

2171575

In [86]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2171575 entries, 0 to 2171574
Data columns (total 3 columns):
post_id    int64
title      object
tag        object
dtypes: int64(1), object(2)
memory usage: 49.7+ MB


In [87]:
data.head()

,post_id,title,tag
0,9,Calculate age in C#,c#
1,16,Filling a DataSet or DataTable from a LINQ que...,c#
2,39,Reliable timer in a console application,c#
3,42,Best way to allow plugins for a PHP application,php
4,59,"How do I get a distinct, ordered list of names...",c#


In [88]:
data["Answer"]= data["title"]

In [89]:
data["Question"]= data["title"]

In [90]:
data["Class"]= data["tag"]

In [91]:
data = data.drop('title', 1) 
data.head()

,post_id,tag,Answer,Question,Class
0,9,c#,Calculate age in C#,Calculate age in C#,c#
1,16,c#,Filling a DataSet or DataTable from a LINQ que...,Filling a DataSet or DataTable from a LINQ que...,c#
2,39,c#,Reliable timer in a console application,Reliable timer in a console application,c#
3,42,php,Best way to allow plugins for a PHP application,Best way to allow plugins for a PHP application,php
4,59,c#,"How do I get a distinct, ordered list of names...","How do I get a distinct, ordered list of names...",c#


In [92]:
data = data.drop('post_id', 1) 
data.head()

,tag,Answer,Question,Class
0,c#,Calculate age in C#,Calculate age in C#,c#
1,c#,Filling a DataSet or DataTable from a LINQ que...,Filling a DataSet or DataTable from a LINQ que...,c#
2,c#,Reliable timer in a console application,Reliable timer in a console application,c#
3,php,Best way to allow plugins for a PHP application,Best way to allow plugins for a PHP application,php
4,c#,"How do I get a distinct, ordered list of names...","How do I get a distinct, ordered list of names...",c#


In [93]:
data = data.drop('tag', 1) 

In [102]:
data.head()


,Answer,Question,Class
0,Calculate age in C#,Calculate age in C#,c#
1,Filling a DataSet or DataTable from a LINQ que...,Filling a DataSet or DataTable from a LINQ que...,c#
2,Reliable timer in a console application,Reliable timer in a console application,c#
4,"How do I get a distinct, ordered list of names...","How do I get a distinct, ordered list of names...",c#
5,Decoding T-SQL CAST in C#/VB.NET,Decoding T-SQL CAST in C#/VB.NET,c#


In [103]:
data=data[data['Class']=='c#']

In [104]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788902 entries, 0 to 2171567
Data columns (total 3 columns):
Answer      788902 non-null object
Question    788902 non-null object
Class       788902 non-null object
dtypes: object(3)
memory usage: 24.1+ MB


In [98]:
data.append(data)

,Answer,Question,Class
0,Calculate age in C#,Calculate age in C#,c#
1,Filling a DataSet or DataTable from a LINQ que...,Filling a DataSet or DataTable from a LINQ que...,c#
2,Reliable timer in a console application,Reliable timer in a console application,c#
4,"How do I get a distinct, ordered list of names...","How do I get a distinct, ordered list of names...",c#
5,Decoding T-SQL CAST in C#/VB.NET,Decoding T-SQL CAST in C#/VB.NET,c#
7,How do I print an HTML document from a web ser...,How do I print an HTML document from a web ser...,c#
8,Adding scripting functionality to .NET applica...,Adding scripting functionality to .NET applica...,c#
10,How do you sort a dictionary by value?,How do you sort a dictionary by value?,c#
15,WinForms ComboBox data binding gotcha,WinForms ComboBox data binding gotcha,c#
19,Robust Random Number Generation,Robust Random Number Generation,c#


In [99]:
data=data.append(data)

In [105]:
 data.head()

,Answer,Question,Class
0,Calculate age in C#,Calculate age in C#,c#
1,Filling a DataSet or DataTable from a LINQ que...,Filling a DataSet or DataTable from a LINQ que...,c#
2,Reliable timer in a console application,Reliable timer in a console application,c#
4,"How do I get a distinct, ordered list of names...","How do I get a distinct, ordered list of names...",c#
5,Decoding T-SQL CAST in C#/VB.NET,Decoding T-SQL CAST in C#/VB.NET,c#


In [106]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788902 entries, 0 to 2171567
Data columns (total 3 columns):
Answer      788902 non-null object
Question    788902 non-null object
Class       788902 non-null object
dtypes: object(3)
memory usage: 24.1+ MB


In [65]:
data=data.append(data)

In [68]:
data=data.append(data)

In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 999
Data columns (total 3 columns):
Answer      4000 non-null object
Question    4000 non-null object
Class       4000 non-null object
dtypes: object(3)
memory usage: 125.0+ KB


In [107]:
data=data[:10000]

In [108]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 31469
Data columns (total 3 columns):
Answer      10000 non-null object
Question    10000 non-null object
Class       10000 non-null object
dtypes: object(3)
memory usage: 312.5+ KB


In [109]:
data.head()

,Answer,Question,Class
0,Calculate age in C#,Calculate age in C#,c#
1,Filling a DataSet or DataTable from a LINQ que...,Filling a DataSet or DataTable from a LINQ que...,c#
2,Reliable timer in a console application,Reliable timer in a console application,c#
4,"How do I get a distinct, ordered list of names...","How do I get a distinct, ordered list of names...",c#
5,Decoding T-SQL CAST in C#/VB.NET,Decoding T-SQL CAST in C#/VB.NET,c#


In [110]:
def pre_process(questions):
    stop_words = stopwords.words("english")
    
    # Remove non english words
    questions = [re.sub('[^a-z(c++)(c#)]', ' ', x.lower()) for x in questions]
    # Tokenlization
    questions_tokens = [nltk.word_tokenize(t) for t in questions]
    # Removing Stop Words
    questions_stop = [[t for t in tokens if (t not in stop_words) and (3 < len(t.strip()) < 15)]
                      for tokens in questions_tokens]
    
    questions_stop = pd.Series(questions_stop)
    return questions_stop

In [111]:
# Initial preprocessing training data
questions = data['Question']
questions_pp = pre_process(questions)

In [112]:
data_tokens = pd.DataFrame({'Question': list(data['Question']),
                            'Question_Tokens': questions_pp,
                            'Answer': list(data['Answer']),
                            'Class': list(data['Class'])
                           })
data_tokens.head()

,Question,Question_Tokens,Answer,Class
0,Calculate age in C#,[calculate],Calculate age in C#,c#
1,Filling a DataSet or DataTable from a LINQ que...,"[filling, dataset, datatable, linq, query, res...",Filling a DataSet or DataTable from a LINQ que...,c#
2,Reliable timer in a console application,"[reliable, timer, console, application]",Reliable timer in a console application,c#
3,"How do I get a distinct, ordered list of names...","[distinct, ordered, list, names, datatable, us...","How do I get a distinct, ordered list of names...",c#
4,Decoding T-SQL CAST in C#/VB.NET,"[decoding, cast]",Decoding T-SQL CAST in C#/VB.NET,c#


### 03-Example

In [83]:
data_example = pd.DataFrame(data_tokens['Question'])
length = data_example['Question'].apply(len)
data_example = data_example.assign(Question_Length=length)
data_example.head()

,Question,Question_Length
0,Calculate age in C#,19
1,Filling a DataSet or DataTable from a LINQ query result set,59
2,Reliable timer in a console application,39
3,Best way to allow plugins for a PHP application,47
4,"How do I get a distinct, ordered list of names from a DataTable using LINQ?",75


In [75]:
# Raw data
example = data_example['Question'][1]
raw_title = 'Raw Data'
raw_result = example
raw_result

'Filling a DataSet or DataTable from a LINQ query result set'

In [76]:
# Remove non english words
re_title = 'Remove non-English Words'
re_result = [re.sub('[^a-z(c++)(c#)]', ' ', x.lower()) for x in pd.Series(example)]
re_result

['filling a dataset or datatable from a linq query result set']

In [77]:
# Tokenlization
tk_title = 'Tokenlization'
tk_result = [nltk.word_tokenize(t) for t in re_result]
print(tk_result)

[['filling', 'a', 'dataset', 'or', 'datatable', 'from', 'a', 'linq', 'query', 'result', 'set']]


In [78]:
# Removing Stop Words
stop_words = stopwords.words("english")
rs_title = 'Removing Stop Words'
rs_result = [[t for t in tokens if (t not in stop_words) and (3 < len(t.strip()) < 15)] for tokens in tk_result]
rs_result

[['filling', 'dataset', 'datatable', 'linq', 'query', 'result']]

In [79]:
data = {'Step' : [raw_title, re_title, tk_title, rs_title],
        'Results' : [raw_result, re_result, tk_result, rs_result]}
df = pd.DataFrame(data)
cols = ['Step', 'Results']
df = df.ix[:,cols]
pd.set_option('display.max_colwidth', 100)
df

,Step,Results
0,Raw Data,Filling a DataSet or DataTable from a LINQ query result set
1,Remove non-English Words,[filling a dataset or datatable from a linq query result set]
2,Tokenlization,"[[filling, a, dataset, or, datatable, from, a, linq, query, result, set]]"
3,Removing Stop Words,"[[filling, dataset, datatable, linq, query, result]]"


### 03-Train Word2Vec

In [113]:
def train_model(train_data):
    """Function trains and creates Word2vec Model using parsed
    data and returns trained model"""
    model = gensim.models.Word2Vec(train_data, min_count=2)
    return model

In [114]:
path=""

In [115]:
dict_language = {'0': 'c#'}

data_tokens['Question_Vectors'] = None
data_tokens['Average_Pooling'] = None
    
for key, value in dict_language.items():
    questions_data = list(data_tokens[data_tokens['Class'] == value]['Question_Tokens'])
    # Train model
    model_name = 'word2vec_model_' + value
    trained_model = train_model(questions_data)
    trained_model.save(model_name)
    print('Saved %s model successfully' % model_name)
    
    # Save Word2Vec model
    word2vec_pickle_path = path + 'stackoverflow_word2vec_' + value + '.bin'
    f = open(word2vec_pickle_path, 'wb')
    pickle.dump(trained_model, f) 
    f.close()
    
    model = gensim.models.KeyedVectors.load(word2vec_pickle_path)
    print("i am in")
    # Calculate the vectors for each question
    for i in range(len(data_tokens)):
        print("ch")
        if data_tokens['Class'][i] == value:
            question_tokens = data_tokens['Question_Tokens'][i]
            question_vectors = []
            for token in question_tokens:
                try:
                    vector = model[token]
                    question_vectors.append(vector)
                except:
                    continue
            # Vectors for each tokens
            data_tokens['Question_Vectors'][i] = question_vectors
            # Average Pooling of all tokens
            data_tokens['Average_Pooling'][i] = list(pd.DataFrame(question_vectors).mean())

Saved word2vec_model_c# model successfully
i am in
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
c

ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
c

ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
c

ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
ch
c

In [76]:
data_tokens['Question_Tokens'] = [" ".join(l) for l in data_tokens['Question_Tokens']]
length = data_tokens['Question_Tokens'].apply(len)
data_tokens = data_tokens.assign(Question_Length=length)
data_tokens.head()

,Question,Question_Tokens,Answer,Class,Question_Vectors,Average_Pooling,Question_Length
0,Calculate age in C#,calculate,Calculate age in C#,c#,"[[0.0046181227, 0.0010197812, -0.0021768531, 0...","[0.004618122708052397, 0.0010197812225669622, ...",9
1,Filling a DataSet or DataTable from a LINQ que...,filling dataset datatable linq query result,Filling a DataSet or DataTable from a LINQ que...,c#,"[[0.0025668305, -0.0026641525, -0.0019031325, ...","[0.0009361894190078601, 0.0003925756609532982,...",43
2,Reliable timer in a console application,reliable timer console application,Reliable timer in a console application,c#,"[[-0.0020239663, -0.00021601502, -0.0007258513...","[-0.00025621961685828865, -0.00012565868019009...",34
3,Best way to allow plugins for a PHP application,best allow plugins application,Best way to allow plugins for a PHP application,php,None,None,30
4,"How do I get a distinct, ordered list of names...",distinct ordered list names datatable using linq,"How do I get a distinct, ordered list of names...",c#,"[[0.0050866045, 0.004084861, 0.0019270796, -0....","[0.0019038520612022175, -4.977736222956862e-06...",48


In [77]:
data_tokens[["Class"]]

,Class
0,c#
1,c#
2,c#
3,php
4,c#
5,c#
6,php
7,c#
8,c#
9,c_cpp


In [116]:
# Export as data as JSON
data_json = json.loads(data_tokens.to_json(orient='records'))

with open(path + 'StackOverflow_Word2Vec.json', 'w') as outfile:
    json.dump(data_json, outfile)

## Word2Vec

In [117]:
try:
    stackoverflow_path = path + 'StackOverflow_Word2Vec.json'

    with open(stackoverflow_path) as file:
        reader = json.load(file)

        classes = []
        questions = []
        questions_tokens = []
        answers = []
        question_lengths = []
        question_vectors = []
        average_pooling = []
        for row in reader:
            classes.append(row['Class'])
            questions.append(row['Question'])
            questions_tokens.append(row['Question_Tokens'].split())
            answers.append(row['Answer'])
            question_lengths.append(row['Question_Length'])
            question_vectors.append(row['Question_Vectors'])
            average_pooling.append(row['Average_Pooling'])

        data_tokens = pd.DataFrame({'Class': classes,
                                    'Question': questions,
                                    'Question_Tokens': questions_tokens,
                                    'Answer': answers,
                                    'Question_Length': question_lengths,
                                    'Question_Vectors': question_vectors,
                                    'Average_Pooling': average_pooling})
except:
    pass

data_tokens.head()

,Question,Question_Tokens,Answer,Class,Question_Vectors,Average_Pooling
0,Calculate age in C#,[calculate],Calculate age in C#,c#,"[[-0.001032434, -0.04862904, -0.021037467, -0....","[-0.0010324340546503663, -0.0486290417611599, ..."
1,Filling a DataSet or DataTable from a LINQ que...,"[filling, dataset, datatable, linq, query, res...",Filling a DataSet or DataTable from a LINQ que...,c#,"[[-0.0008110215, -0.030568374, -0.012533399, -...","[-0.022897001773041364, -0.21079957578331232, ..."
2,Reliable timer in a console application,"[reliable, timer, console, application]",Reliable timer in a console application,c#,"[[-0.0035664532, -0.014789825, -0.006310183, -...","[-0.02002632326912135, -0.17745322501286864, -..."
3,"How do I get a distinct, ordered list of names...","[distinct, ordered, list, names, datatable, us...","How do I get a distinct, ordered list of names...",c#,"[[5.147505e-05, -0.03852874, -0.014045121, -0....","[-0.026128292793665815, -0.24859928339719772, ..."
4,Decoding T-SQL CAST in C#/VB.NET,"[decoding, cast]",Decoding T-SQL CAST in C#/VB.NET,c#,"[[0.0008616242, -0.0073377583, -0.0035088568, ...","[-0.0055029937357176095, -0.07605473417788744,..."


In [118]:
# Greeting function
GREETING_INPUTS = ("hello", "hi", "greetings", "hello i need help", "good day","hey","i need help", "greetings")
GREETING_RESPONSES = ["Good day, How may i of help?", "Hello, How can i help?", "hello", "I am glad! You are talking to me."]
           
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [119]:
def Talk_To_Javris(data_language, model):
    
    # Preprocessing of user input
    sentence_pp = pre_process(pd.Series(sentence)) 

    cosines = []
    try:
        # Get vectors and average pooling
        question_vectors = []
        for token in sentence_pp:
            try:
                vector = model[token]
                question_vectors.append(vector)
            except:
                continue
        question_ap = list(pd.DataFrame(question_vectors[0]).mean())

        # Calculate cosine similarity
        for t in data_language['Average_Pooling']:
            if t is not None and len(t) == len(question_ap):
                val = cosine_similarity([question_ap], [t])
                cosines.append(val[0][0])
            else:
                cosines.append(0)
    except:
        pass
            
    # If not in the topic trained
    if len(cosines) == 0:
        not_understood = "Apology, I do not understand. Can you rephrase?"
        return not_understood, 999
    
    else: 
        # Sort similarity
        index_s =[]
        score_s = []
        for i in range(len(cosines)):
            x = cosines[i]
            if x >= 0.9:
                index_s.append(i)
                score_s.append(cosines[i])

        reply_indexes = pd.DataFrame({'index': index_s, 'score': score_s})
        reply_indexes = reply_indexes.sort_values(by="score" , ascending=False)

        # Find Top Questions and Score
        r_index = int(reply_indexes['index'].iloc[0])
        r_score = float(reply_indexes['score'].iloc[0])

        reply = str(data_language.iloc[:,0][r_index])
        
        return reply, r_score

In [ ]:
flag_language = True
flag_query = True
dict_language = {'0': 'python', '1': 'c++', '2': 'c#', '3': 'java', '4': 'ios', '5': 'android', '6': 'html', 
                 '7': 'jquery', '8': 'php', '9': 'javascript'}

print('......................................................................................')
print('\x1b[1;37;40m' + 'Jarvis' + '\x1b[0m' + ': ' + 'My name is Jarvis, a Programming Language Apprentice Bot.')
print('\x1b[1;37;40m' + 'Jarvis' + '\x1b[0m' + ': ' + 'I will try my best to answer your query.')
print('\x1b[1;37;40m' + 'Jarvis' + '\x1b[0m' + ': ' + 'If you want to exit, you can type < bye >.')

while(flag_language == True):
    print("......................................................................................")
    print('\x1b[1;37;40m' + 'Jarvis' + '\x1b[0m' + ': ' + 'Please select which language you want to enquire, ' +
      'you can type:')
    print('\x1b[1;37;40m' + 'Jarvis' + '\x1b[0m' + ': ' + '< 0 > for python     < 1 > for c++      < 2 > for c#')
    print('\x1b[1;37;40m' + 'Jarvis' + '\x1b[0m' + ': ' + '< 3 > for java       < 4 > for ios      < 5 > for android')
    print('\x1b[1;37;40m' + 'Jarvis' + '\x1b[0m' + ': ' + '< 6 > for html       < 7 > for jquery   < 8 > for php')
    print('\x1b[1;37;40m' + 'Jarvis' + '\x1b[0m' + ': ' + '< 9 > for javascript')
    print("......................................................................................")
    sentence = input('\x1b[0;30;47m' + 'USER  ' + '\x1b[0m' + ':')
    print("......................................................................................")
    
    if(sentence.lower() != 'bye'):
        if (sentence.lower() in list(dict_language.keys())):
            language = dict_language[sentence.lower()]
            data_language = data_tokens[data_tokens['Class'] == language]
            data_language = pd.DataFrame({'Question': list(data_language['Question']),
                                          'Question_Tokens': list(data_language['Question_Tokens']),
                                          'Answer': list(data_language['Answer']),
                                          'Class': list(data_language['Class']),
                                          'Question_Vectors': list(data_language['Question_Vectors']),
                                          'Average_Pooling': list(data_language['Average_Pooling'])
                                         })
            
            # Read word2vec model
            word2vec_pickle_path = path + 'stackoverflow_word2vec_' + language + '.bin'
            model = gensim.models.KeyedVectors.load(word2vec_pickle_path)
            
            flag_language = False
            flag_query = True
    else:
        flag_language = False
        flag_query = False

print("......................................................................................")
print('\x1b[1;37;40m' + 'Jarvis' + '\x1b[0m' + ': ' + 'Let''s start! Please input your question now.')
    
while(flag_query == True):
    print("......................................................................................")
    sentence = input('\x1b[0;30;47m' + 'USER  ' + '\x1b[0m' + ':')
    print("......................................................................................")

    if(sentence.lower() != 'bye'):
        if(greeting(sentence.lower()) != None):
            print('\x1b[1;37;40m' + 'JARVIS' + '\x1b[0m' + ': ' + greeting(sentence.lower()))
        else:
            reply, score = Talk_To_Javris(data_language, model)
            print('\x1b[1;37;40m' + 'JARVIS'+'\x1b[0m'+': '+reply)

            #For Tracing, comment to remove from print 
            #print("")
            #print("SCORE: " + str(score))
    else:
        flag_query = False
print('\x1b[1;37;40m' + 'JARVIS' + '\x1b[0m' + ': ' + 'Bye! Hope that i am of help.') 

......................................................................................
Jarvis: My name is Jarvis, a Programming Language Apprentice Bot.
Jarvis: I will try my best to answer your query.
Jarvis: If you want to exit, you can type < bye >.
......................................................................................
Jarvis: Please select which language you want to enquire, you can type:
Jarvis: < 0 > for python     < 1 > for c++      < 2 > for c#
Jarvis: < 3 > for java       < 4 > for ios      < 5 > for android
Jarvis: < 6 > for html       < 7 > for jquery   < 8 > for php
Jarvis: < 9 > for javascript
......................................................................................
USER  :2
......................................................................................
......................................................................................
Jarvis: Lets start! Please input your question now.
................................................